In [ ]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv()
# Use environment variables for sensitive information
mongo_connection_string = ""
mongo_client = MongoClient(mongo_connection_string)

database_name = "menuDB"
db = mongo_client[database_name]
collection_name = "DisheCollection"
index_name = "DisheIndex"
collection = db[collection_name]

C:\Users\ramya.chandrasekaran\AppData\Local\Temp\ipykernel_24604\4067360896.py:7: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = MongoClient(mongo_connection_string)
c:\Saravanan\code-practise\AI-ML\sample-codes\.venv\Lib\site-packages\pymongo\synchronous\mongo_client.py:1737: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


In [ ]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import Runnable

openai_embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME", "text-embedding-ada-002")
openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME", "text-embedding-ada-002")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  # Make sure this is set in your environment
api_key= os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://myragpocai-resource.openai.azure.com/",
    model="text-embedding-ada-002",
    azure_deployment="text-embedding-ada-002",
    api_key=""
)

In [13]:
from langchain_community.vectorstores import AzureCosmosDBVectorSearch
from langchain_core.documents import Document
import json
# Load JSON data
with open('sample_menu_100.json', 'r') as file:
    raw_data = json.load(file)

# Convert raw data to list of Document objects
documents = [
    Document(
        page_content=item.get("text") or json.dumps(item),  # fallback to full dict if "text" not available
        metadata={k: v for k, v in item.items() if k != "text"}
    )
    for item in raw_data
]

# Now create the vector store
vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_documents(
    documents=documents,
    embedding=azure_openai_embeddings,
    collection=collection,
    index_name=index_name,
)

In [ ]:
from azure.cosmos.aio.cosmos_client_connection import CosmosDBSimilarityType, CosmosDBVectorSearchType

num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_HNSW
m = 16
ef_construction = 64

vector_store.create_index(
    num_lists, dimensions, similarity_algorithm, kind, m, ef_construction
)

In [ ]:
vector_store: AzureCosmosDBVectorSearch =  AzureCosmosDBVectorSearch.from_connection_string(
    connection_string=mongo_connection_string,
    namespace=f"{database_name}.{collection_name}",
    embedding=azure_openai_embeddings,
)

In [ ]:
query = "beef dishes"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

In [ ]:
openai_chat_model = os.getenv("AZURE_OPENAI_CHAT_MODEL_NAME", "gpt-35-turbo")
openai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-35-turbo")
azure_openai_chat: AzureChatOpenAI = AzureChatOpenAI(
    model=openai_chat_model,
    azure_deployment=openai_chat_deployment,
)

In [ ]:
history_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            """Given the above conversation,
            generate a search query to look up to get information relevant to the conversation""",
        ),
    ]
)

context_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's questions based on the below context:\n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

In [ ]:
search_type = "similarity"
limit = 3
score_threshold = 0.7
vector_store_retriever = vector_store.as_retriever(
    search_type=search_type, search_kwargs={"k": limit, "score_threshold": score_threshold}
)

In [ ]:
retriever_chain = create_history_aware_retriever(azure_openai_chat, vector_store_retriever, history_prompt)

In [ ]:
context_chain = create_stuff_documents_chain(llm=azure_openai_chat, prompt=context_prompt)

In [ ]:
rag_chain: Runnable = create_retrieval_chain(
    retriever=retriever_chain,
    combine_docs_chain=context_chain,
)